### 2. Вірусні новини

У статті [Automatic Extraction of News Values from Headline Text](http://www.aclweb.org/anthology/E17-4007) описано основні ознаки заголовків, які кидаються в очі і змушують читача таки прочитати новину:
1. наявність імен людей, назв компаній тощо
2. емоційне забарвлення
3. ступені порівняння
4. близькість
5. елемент несподіванки
6. унікальність

**Завдання:**
1. Напишіть програму, яка аналізує заголовок за першими трьома ознаками (у спрощеній формі)
   * Чи є в заголовку іменовані стуності?
   * Чи є заголовок позитивно чи негативно забарвлений?
   * Чи є в заголовку прикметники та прислівники вищого і найвищого ступенів порівняння?
2. Проженіть вашу програму на [корпусі заголовків з The Examiner](examiner-headlines.txt). Для кожної з трьох ознак, визначте відсоток заголовків у корпусі, які її мають.
3. Збережіть програму та пораховану статистику в директорії з вашим іменем.

Додаткова інформація:
- Типи сутностей, які впливають на "вірусність" заголовка, виберіть самостійно.
- Для визначення емоційного забарвлення, використайте [SentiWordNet](http://sentiwordnet.isti.cnr.it/). Наприклад, можна перевірити, що середнє значення позитивності/негативності слова у заголовку перевищує 0.5. Для визначення середнього значення можна брати до п'яти перших значень слова з такою частиною мови. Будьте креативними та експериментуйте.


#### Sentiment

In [60]:
def apply_stats(select_fun):
    with open('examiner-headlines.txt') as headlines:
        n = 0
        m = 0
        for headline in headlines:
            m += select_fun(headline)
            n += 1
        return (n, m, (m * 100) / n)

In [2]:
import spacy

nlp = spacy.load('en_core_web_md')

In [32]:
VIRAL_NER = {'PERSON', 'NORP', 'FAC', 'ORG', 'GPE', 'LOC', 
             'PRODUCT', 'EVENT', 'WORK_OF_ART', 'LAW', 'LANGUAGE', 'MONEY'}

def contains_viral_ner(headline):
    doc = nlp(headline)
    for ent in doc.ents:
        if ent.label_ in VIRAL_NER:
            return True
    return False

In [61]:
total, ner_num, pct = apply_stats(contains_viral_ner)
print(f"Headlines total: {total}")
print("Headlines that contain viral named entities: {} ({:.2f}%)".format(ner_num, pct))

Headlines total: 5000
Headlines that contain viral named entities: 3646 (72.92%)


In [63]:
SUPERLATIVES = {'JJR', 'JJS', 'RBR', 'RBS'}

def contains_superlatives(headline):
    doc = nlp(headline)
    for token in doc:
        if token.tag_ in SUPERLATIVES:
            return True
    return False    

In [64]:
total, super_num, pct = apply_stats(contains_superlatives)
print(f"Headlines total: {total}")
print("Headlines that contain superlatives: {} ({:.2f}%)".format(super_num, pct))

Headlines total: 5000
Headlines that contain superlatives: 232 (4.64%)


In [67]:
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
 
def penn_to_wordnet(tag):
    
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    
    return None
 
  
def viral_sentiment(headline):
    
    sentiment = 0.0
    tokens_count = 0    
    
    tokens = nlp(headline)
        
    for token in tokens:      
        
        wn_tag = penn_to_wordnet(token.tag_)
        
        if not wn_tag:
            continue

        synsets = wn.synsets(token.lemma_, pos=wn_tag)        
        
        if not synsets:
            continue
        
        word_sentiment = 0.0
        meaning_num = 0
        
        for synset in synsets[:5]:
            swn_synset = swn.senti_synset(synset.name())            

            word_sentiment += swn_synset.pos_score() - swn_synset.neg_score()
            meaning_num += 1
            
        sentiment += word_sentiment / meaning_num    
         
    if sentiment > 0.5 or sentiment < -0.5:
        return True
        
    return False

In [68]:
total, senti_num, pct = apply_stats(viral_sentiment)
print(f"Headlines total: {total}")
print("Headlines that have viral sentiment: {} ({:.2f}%)".format(senti_num, pct))

Headlines total: 5000
Headlines that have viral sentiment: 640 (12.80%)


In [69]:
with open('examiner-headlines.txt') as headlines:
    with open('viral-sentiment.txt', 'w') as senti_output:
        with open('ner-headlines.txt', 'w') as ner_output:
            with open('superlative-headlines.txt', 'w') as sup_output:
                for headline in headlines:
                    if viral_sentiment(headline):
                        senti_output.write(headline)
                    if contains_viral_ner(headline):
                        ner_output.write(headline)
                    if contains_superlatives(headline):
                        sup_output.write(headline)